# Preparing the environment and installing libraries:

In [2]:
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq
from peft import LoraConfig, get_peft_model
import json

2025-03-14 15:59:09.026899: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-14 15:59:09.245743: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-14 15:59:10.185088: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-14 15:59:12.697656: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Model Fine-tuning

### 1. Prepare the data

After the preparation, the dataset looks like this:
```json
{
  "title": "Document title...",
  "content": "Original document text...",
  "chosen": "Generated summary with the big model...",
  "rejected": "Generated summary with the smaller model..."
}
```
And it's split into `train` and `test` sets.

In [4]:
token_size = "2k" # 8k

In [5]:
train_dataset = json.load(open(f"data_{token_size}_tokens/train.json"))
test_dataset = json.load(open(f"data_{token_size}_tokens/test.json"))

train_dataset = Dataset.from_list(train_dataset)
test_dataset = Dataset.from_list(test_dataset)

In [6]:
print(train_dataset[0]["chosen"])

"Dans les prisons de Nantes" est une chanson populaire en France et à l'étranger, chantée par de nombreux artistes. Originaire de traditions françaises anciennes, elle raconte l'histoire d'un prisonnier et d'une fille du geôlier. Cette chanson évoque des prisons nantaises historiques, notamment celle de l'Entrepôt des cafés pendant la Révolution française. Sa popularité a été renforcée par des versions célèbres d'artistes comme Tri Yann et Nolwenn Leroy. <sentence/>


# 2. Custom Quantization

In [7]:
# Configure 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
)

# 3. Model Definition

There are several models interesting to fine-tune for this task. We will use the `transformers` library to fine-tune a model from the list.

In [12]:
model_name = "Qwen/Qwen2.5-0.5B-Instruct"

# Load the model in 4-bit
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map="auto")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

Now we will use the tokenizer to encode the input text.

In [13]:
def tokenize_function(examples):
    # Add EOS token manually to both content and summary
    full_prompt = f"<human>: Résume {examples['content']}\n<assistant>: {examples['chosen']}"

    # Tokenize the full prompt
    tokenized_full_prompt = tokenizer(full_prompt, return_tensors="pt")
    
    # Clone the tokenized input to create labels
    labels = tokenized_full_prompt["input_ids"].clone()

    prompt = full_prompt[:full_prompt.find("<assistant>")] + "<assistant>:"

    end_prompt_index = tokenizer(prompt, return_tensors="pt").input_ids.shape[1]

    labels[:, :end_prompt_index] = -100

    return {
        'input_ids': tokenized_full_prompt["input_ids"].flatten(),
        'labels': labels.flatten(),
        'attention_mask': tokenized_full_prompt["attention_mask"].flatten(),
    }

# Tokenize the dataset

data = train_dataset.map(tokenize_function)


Map: 100%|██████████| 4589/4589 [00:10<00:00, 445.42 examples/s]


We will use Lora to fine-tune the model efficiently without having to adjust all the parameters.

In [14]:
# Extract target_modules
target_modules = [
    'gate_proj',
    'up_proj',
    'down_proj',
]

config = LoraConfig(
    r=32,
    lora_alpha=64,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=target_modules,
)


model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 13,271,040 || all params: 507,303,808 || trainable%: 2.6160


In [19]:
def summarize_document(document, model=model, tokenizer=tokenizer):
    # Tokenize the input document
    prompt = f"<human>: Summarize the following text :\n\n{document}\n\nYou should respond In French, with 5 sentences maximum.\n\n<assistant>:"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generate summary
    summary_ids = model.generate(**inputs, max_new_tokens=128)

    # Decode the summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    assistant_index = summary.find("<assistant>:")
    if assistant_index != -1:
        summary = summary[assistant_index + len("<assistant>:"):].strip()

    return summary

In [17]:
article = """Le Monument-Lefebvre est un lieu historique national du Canada situé à Memramcook, au Nouveau-Brunswick.\n\n\n== Histoire ==\nLe Monument-Lefebvre est inauguré les 16 et 17 juin 1897 et nommé ainsi en l'honneur du père Camille Lefebvre, fondateur du Collège Saint-Joseph, décédé deux ans auparavant.\nIl est tout d'abord utilisé comme laboratoire par le Collège Saint-Joseph, tandis que son amphithéâtre permet d'organiser des concerts, des conférences et de nombreuses manifestations diverses. Le lieu et son fondateur ont joué un rôle déterminant dans la renaissance acadienne.\nAprès la fermeture du Collège en 1972, le Monument-Lefebvre manque de disparaître mais est finalement rénové et déclaré lieu historique national en 1994.\n\n\n== Architecture ==\nLe Monument-Lefebvre est construit en grès rustiqué de couleur olive. Sa façade symétrique comprend des éléments néoromans. Il abrite un théâtre et des salle de classe.\n\n\n== Notes et références ==\n\n\n== Liens externes ==\nRessources relatives à l'architecture : Édifices fédéraux patrimoniaux Lieux historiques nationaux Répertoire canadien des lieux patrimoniaux \n(fr + en) Site officiel\n\n Portail de l’Acadie   Portail du Nouveau-Brunswick   Portail des lieux patrimoniaux du Canada"""

summarize_document(article)

"Le Monument-Lefebvre est un lieu historique national du Canada situé à Memramcook, au Nouveau-Brunswick. Initialement utilisé comme laboratoire par le Collège Saint-Joseph, il a été déclaré lieu historique national en 1994 après la fermeture du Collège en 1972. Il abrite un théâtre et une salle de classe et est aujourd'hui rénové pour accueillir des événements divers. Il a joué un rôle déterminant dans la renaissance acadienne. Parallèlement, il est"

In [33]:
training_args = TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=32,
    num_train_epochs=1,
    learning_rate=2e-4,
    bf16=True,
    save_total_limit=3,
    logging_steps=20,
    output_dir="results",
    max_steps=500,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    label_names=["labels"],
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [34]:
trainer = Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model),
)

model.config.use_cache = False
trainer.train()

Step,Training Loss
20,0.909900
40,0.919300
60,0.973800
80,1.006500
100,1.020900
120,1.005200
140,1.002600
160,0.804800
180,0.769600
200,0.778600


TrainOutput(global_step=500, training_loss=0.7453178424835205, metrics={'train_runtime': 3151.5771, 'train_samples_per_second': 5.077, 'train_steps_per_second': 0.159, 'total_flos': 4.016179539972557e+16, 'train_loss': 0.7453178424835205, 'epoch': 3.499560246262093})

# 4. Test the fine-tuned model

In [21]:
%%time

article = """Le Monument-Lefebvre est un lieu historique national du Canada situé à Memramcook, au Nouveau-Brunswick.\n\n\n== Histoire ==\nLe Monument-Lefebvre est inauguré les 16 et 17 juin 1897 et nommé ainsi en l'honneur du père Camille Lefebvre, fondateur du Collège Saint-Joseph, décédé deux ans auparavant.\nIl est tout d'abord utilisé comme laboratoire par le Collège Saint-Joseph, tandis que son amphithéâtre permet d'organiser des concerts, des conférences et de nombreuses manifestations diverses. Le lieu et son fondateur ont joué un rôle déterminant dans la renaissance acadienne.\nAprès la fermeture du Collège en 1972, le Monument-Lefebvre manque de disparaître mais est finalement rénové et déclaré lieu historique national en 1994.\n\n\n== Architecture ==\nLe Monument-Lefebvre est construit en grès rustiqué de couleur olive. Sa façade symétrique comprend des éléments néoromans. Il abrite un théâtre et des salle de classe.\n\n\n== Notes et références ==\n\n\n== Liens externes ==\nRessources relatives à l'architecture : Édifices fédéraux patrimoniaux Lieux historiques nationaux Répertoire canadien des lieux patrimoniaux \n(fr + en) Site officiel\n\n Portail de l’Acadie   Portail du Nouveau-Brunswick   Portail des lieux patrimoniaux du Canada"""

summarize_document(article)

CPU times: user 1.87 s, sys: 916 µs, total: 1.87 s
Wall time: 1.87 s


"Le Monument-Lefebvre est un lieu historique national du Canada situé à Memramcook, au Nouveau-Brunswick. Il a été inauguré les 16 et 17 juin 1897 et est aujourd'hui une destination touristique. Il a été utilisée comme laboratoire par le Collège Saint-Joseph, puis a obtenu son statut de monument historique national en 1994. Ses structures sont composées de gravures, de sculptures et de murs sculptés. C'est une des plus anciennes monuments historiques du Canada, avec une histoire de près"

# 3. Save the fine-tuned model

In [39]:
# save the model
finetunedmodel = model.merge_and_unload()
finetunedmodel.save_pretrained("models/2k_SFT_finetuned")
tokenizer.save_pretrained("models/2k_SFT_finetuned")

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 896)
        (layers): ModuleList(
          (0-23): 24 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): Linear4bit(in_features=896, out_features=896, bias=True)
              (k_proj): Linear4bit(in_features=896, out_features=128, bias=True)
              (v_proj): Linear4bit(in_features=896, out_features=128, bias=True)
              (o_proj): Linear4bit(in_features=896, out_features=896, bias=False)
            )
            (mlp): Qwen2MLP(
              (gate_proj): Linear4bit(in_features=896, out_features=4864, bias=False)
              (up_proj): Linear4bit(in_features=896, out_features=4864, bias=False)
              (down_proj): Linear4bit(in_features=4864, out_features=896, bias=False)
              (act_fn): SiLU()
            )
            (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
 